In [1]:
#SQL Mini Project- SpringBoard

In [3]:
#Q1: Some of the facilities charge a fee to members, but some do not.
#Please list the names of the facilities that do. 

SELECT name FROM Facilities
WHERE member cost > 0

In [5]:
# Q2: How many facilities do not charge a fee to members? 

SELECT count(*) FROM Facilities WHERE member cost = 0

In [6]:
#Q3: How can you produce a list of facilities that charge a fee to members,
#where the fee is less than 20% of the facility's monthly maintenance cost?
#Return the facid, facility name, member cost, and monthly maintenance of the
#facilities in question.

SELECT facid, name, membercost, monthlymaintenance 
FROM Facilities 
WHERE membercost > 0 AND membercost / monthlymaintenance < 0.2

In [7]:
#Q4: How can you retrieve the details of facilities with ID 1 and 5?
#Write the query without using the OR operator.

SELECT * FROM facilities WHERE facid IN (1, 5);

In [8]:
#Q5: How can you produce a list of facilities, with each labelled as
#'cheap' or 'expensive', depending on if their monthly maintenance cost is
#more than $100? Return the name and monthly maintenance of the facilities
#in question. 

SELECT name, monthlymaintenance, 
CASE WHEN monthlymaintenance > 100 THEN 'expensive'
     ELSE 'cheap' END AS cheap_or_expensive_col
FROM Facilities

In [9]:
#Q6: You'd like to get the first and last name of the last member(s)
#who signed up. Do not use the LIMIT clause for your solution.

Select firstname, surname 
FROM Members 
WHERE join date = (select MAX(join date) FROM Members)

In [10]:
#Q7: How can you produce a list of all members who have used a tennis court?
#Include in your output the name of the court, and the name of the member
#formatted as a single column. Ensure no duplicate data, and order by
#the member name.

SELECT DISTINCT 
    CONCAT(memsDB.firstname, ' ' , memsDB.surname) as member,
    facsDB.name as Facility
From Members memsDB
    Inner Join Bookings booksDB ON booksDB.memid = memsDB.memid
    Inner Join Facilities facsDB ON booksDB.facid = facsDB.facid
WHERE facsDB.name  LIKE ‘Tennis Court%’
Order by member

In [11]:
#Q8: How can you produce a list of bookings on the day of 2012-09-14 which
#will cost the member (or guest) more than $30? Remember that guests have
#different costs to members (the listed costs are per half-hour 'slot'), and
#the guest user's ID is always 0. Include in your output the name of the
#facility, the name of the member formatted as a single column, and the cost.
#Order by descending cost, and do not use any subqueries.

SELECT facsDB.name Facility_Name, CONCAT(memsDB.firstname, ' ' , memsDB.surname) as member, 
CASE 
    WHEN memsDB.memid = 0 THEN facsDB.guestcost * booksDB.slots 
    ELSE  facsDB.membercost * booksDB.slots END as Cost
FROM Members memsDB
    INNER Join Bookings booksDB ON memsDB.memid = booksDB.memid
    INNER JOIN Facilities facsDB ON booksDB.facid = facsDB.facid
WHERE 
    booksDB.starttime >='2012-09-14' and 
    booksDB.starttime < '2012-09-15' and 
        (memsDB.memid = 0 and booksDB.slots*facsDB.guestcost > 30) or 
        (memsDB.memid != 0 and booksDB.slots*facsDB.membercost > 30))
Order By Cost desc 

In [12]:
#Q9: This time, produce the same result as in Q8, but using a subquery.

SELECT bookings.member, bookings.facility_name, bookings.cost
FROM (
    SELECT facsDB.name Facility_Name, CONCAT( memsDB.firstname,  ' ', memsDB.surname ) AS member, 
    CASE 
        WHEN memsDB.memid =0
        THEN facsDB.guestcost * booksDB.slots
        ELSE facsDB.membercost * booksDB.slots
    END AS Cost
    FROM Members memsDB
        INNER JOIN Bookings booksDB ON memsDB.memid = booksDB.memid
        INNER JOIN Facilities facsDB ON booksDB.facid = facsDB.facid
        WHERE booksDB.starttime >=  '2012-09-14'
        AND booksDB.starttime <  '2012-09-15'
    ) AS bookings
WHERE cost >30
ORDER BY cost DESC

In [13]:
#Q10: Produce a list of facilities with a total revenue less than 1000.
#The output of facility name and total revenue, sorted by revenue. Remember
#that there's a different cost for guests and members! 

SELECT facsDB.name, SUM(CASE
                        WHEN memid = 0 THEN slots*guestcost
                        ELSE slots*membercost END) as revenue
FROM Bookings booksDB
INNER JOIN Facilities facsDB ON booksDB.facid = facsDB.facid
GROUP BY facsDB.name
HAVING SUM(CASE
            WHEN memid = 0 THEN slots*facsDB.guestcost
            ELSE slots*membercost END) < 1000
ORDER BY revenue 
